In this notebook Longitude and latitude columns are added in order for further visualisations and processing. This proccess take a lot of time, therefore it is done in separate notebook and columns added before all further manipulation with the dataset.

In [43]:
from geopy.geocoders import Nominatim
import pandas as pd
import time

In [44]:
df = pd.read_parquet("../data/raw/pollution_dataset.parquet", engine="pyarrow")
df.head()

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


In [45]:
# Use only columns needed for geocoding
unique_addresses = df[["Address", "City", "State"]].drop_duplicates().copy()
print("Unique Address combinations:", len(unique_addresses))


Unique Address combinations: 204


In [46]:
def clean_address(a):
    if not isinstance(a, str):
        return ""
    a = a.replace("STN", "")
    a = a.replace("-", " ")
    a = a.replace("  ", " ")
    return a.strip()

unique_addresses["CleanAddress"] = unique_addresses["Address"].apply(clean_address)

# Build full postal-style address
unique_addresses["FullAddress"] = (
    unique_addresses["CleanAddress"] + ", " +
    unique_addresses["City"].fillna("") + ", " +
    unique_addresses["State"].fillna("")
).str.strip()


In [47]:
unique_addresses.head()


,Address,City,State,CleanAddress,FullAddress
0,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Phoenix,Arizona,1645 E ROOSEVELT ST CENTRAL PHOENIX,"1645 E ROOSEVELT ST CENTRAL PHOENIX, Phoenix, ..."
1396,2857 N MILLER RD-S SCOTTSDALE STN,Scottsdale,Arizona,2857 N MILLER RD S SCOTTSDALE,"2857 N MILLER RD S SCOTTSDALE, Scottsdale, Ari..."
2128,"1237 S. BEVERLY , TUCSON",Tucson,Arizona,"1237 S. BEVERLY , TUCSON","1237 S. BEVERLY , TUCSON, Tucson, Arizona"
3516,2956-A TREAT BOULEVARD,Concord,California,2956 A TREAT BOULEVARD,"2956 A TREAT BOULEVARD, Concord, California"
4968,5551 BETHEL ISLAND RD,Bethel Island,California,5551 BETHEL ISLAND RD,"5551 BETHEL ISLAND RD, Bethel Island, California"


In [48]:
geolocator = Nominatim(user_agent="aqi_geocoder_app")
def geocode_address(address):
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return pd.Series([location.latitude, location.longitude])
        else:
            return pd.Series([None, None])
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return pd.Series([None, None])

In [49]:
latitudes = []
longitudes = []

for i, addr in enumerate(unique_addresses["FullAddress"]):
    
    lat, lon = geocode_address(addr)
    latitudes.append(lat)
    longitudes.append(lon)

    if i % 10 == 0:
        print(f"{i}/{len(unique_addresses)} geocoded...")

    time.sleep(1)  # absolutely required for Nominatim


0/204 geocoded...
10/204 geocoded...
20/204 geocoded...
30/204 geocoded...
40/204 geocoded...
50/204 geocoded...
60/204 geocoded...
70/204 geocoded...
80/204 geocoded...
90/204 geocoded...
100/204 geocoded...
110/204 geocoded...
120/204 geocoded...
130/204 geocoded...
140/204 geocoded...
150/204 geocoded...
160/204 geocoded...
170/204 geocoded...
180/204 geocoded...
190/204 geocoded...
200/204 geocoded...


In [50]:
unique_addresses["Latitude"] = latitudes
unique_addresses["Longitude"] = longitudes


In [51]:
unique_addresses.to_parquet("../data/raw/coordinates.parquet", index=False)
print("Saved coordinates.parquet")


Saved coordinates.parquet


In [52]:
unique_addresses.head()
# print(unique_addresses.isna().sum())
# print(unique_addresses.shape)

,Address,City,State,CleanAddress,FullAddress,Latitude,Longitude
0,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Phoenix,Arizona,1645 E ROOSEVELT ST CENTRAL PHOENIX,"1645 E ROOSEVELT ST CENTRAL PHOENIX, Phoenix, ...",33.458080,-112.046091
1396,2857 N MILLER RD-S SCOTTSDALE STN,Scottsdale,Arizona,2857 N MILLER RD S SCOTTSDALE,"2857 N MILLER RD S SCOTTSDALE, Scottsdale, Ari...",NaN,NaN
2128,"1237 S. BEVERLY , TUCSON",Tucson,Arizona,"1237 S. BEVERLY , TUCSON","1237 S. BEVERLY , TUCSON, Tucson, Arizona",32.206263,-110.879259
3516,2956-A TREAT BOULEVARD,Concord,California,2956 A TREAT BOULEVARD,"2956 A TREAT BOULEVARD, Concord, California",37.947731,-122.008390
4968,5551 BETHEL ISLAND RD,Bethel Island,California,5551 BETHEL ISLAND RD,"5551 BETHEL ISLAND RD, Bethel Island, California",38.011784,-121.641080


In [53]:
print(unique_addresses.isna().sum())
print(unique_addresses.shape)

Address           0
City              0
State             0
CleanAddress      0
FullAddress       0
Latitude        115
Longitude       115
dtype: int64
(204, 7)


In [54]:
failed = unique_addresses[unique_addresses["Latitude"].isna()]
failed

,Address,City,State,CleanAddress,FullAddress,Latitude,Longitude
1396,2857 N MILLER RD-S SCOTTSDALE STN,Scottsdale,Arizona,2857 N MILLER RD S SCOTTSDALE,"2857 N MILLER RD S SCOTTSDALE, Scottsdale, Ari...",NaN,NaN
6384,"UNIT 759 EL PORTAL SHOPPING CENTER, San Pablo",San Pablo,California,"UNIT 759 EL PORTAL SHOPPING CENTER, San Pablo","UNIT 759 EL PORTAL SHOPPING CENTER, San Pablo,...",NaN,NaN
9300,"1029 ETHEL ST, CALEXICO HIGH SCHOOL",Calexico,California,"1029 ETHEL ST, CALEXICO HIGH SCHOOL","1029 ETHEL ST, CALEXICO HIGH SCHOOL, Calexico,...",NaN,NaN
19074,"7823 BLACKFOOT WAY, NORTH HIGHLANDS",Not in a city,California,"7823 BLACKFOOT WAY, NORTH HIGHLANDS","7823 BLACKFOOT WAY, NORTH HIGHLANDS, Not in a ...",NaN,NaN
20182,"DEL PASO-2701 AVALON DR, SACRAMENTO",Arden-Arcade,California,"DEL PASO 2701 AVALON DR, SACRAMENTO","DEL PASO 2701 AVALON DR, SACRAMENTO, Arden-Arc...",NaN,NaN
...,...,...,...,...,...,...,...
1541073,"Piney Run, Frostburg Reservoir, Finzel",Grantsville,Maryland,"Piney Run, Frostburg Reservoir, Finzel","Piney Run, Frostburg Reservoir, Finzel, Grants...",NaN,NaN
1548535,150 Pillsbury Rd,Not in a city,New Hampshire,150 Pillsbury Rd,"150 Pillsbury Rd, Not in a city, New Hampshire",NaN,NaN
1568833,NARSTO SITE - ARENDTSVILLE,Not in a city,Pennsylvania,NARSTO SITE ARENDTSVILLE,"NARSTO SITE ARENDTSVILLE, Not in a city, Penn...",NaN,NaN
1594263,"90 GAS HILL ROAD, RIVERTON, WY",Riverton,Wyoming,"90 GAS HILL ROAD, RIVERTON, WY","90 GAS HILL ROAD, RIVERTON, WY, Riverton, Wyoming",NaN,NaN


In [55]:
"""
df = pd.merge(df, unique_addresses, on="Address", how="left")
df.to_parquet("../data/preprocessed/pollution_dataset_enriched.parquet", index=False)
"""

'\ndf = pd.merge(df, unique_addresses, on="Address", how="left")\ndf.to_parquet("../data/preprocessed/pollution_dataset_enriched.parquet", index=False)\n'